In [1]:
import pandas as pd
import math
from random import * 
# import data sets that will be used
seeds = pd.read_csv("../csvs/kaggle/predictive/NCAATourneySeeds.csv")
reg_season = pd.read_csv("../csvs/kaggle/predictive/RegularSeasonDetailedResults.csv")
regions = pd.read_csv("../csvs/kaggle/predictive/Seasons.csv")
team_names = pd.read_csv("../csvs/kaggle/predictive/Teams.csv")
outcomes_14 = pd.read_csv("../csvs/1314/1314_outcomes.csv", encoding = 'latin-1')
outcomes_15 = pd.read_csv("../csvs/1415/1415_outcomes.csv", encoding = 'latin-1')
outcomes_16 = pd.read_csv("../csvs/1516/1516_outcomes.csv", encoding = 'latin-1')
outcomes_17 = pd.read_csv("../csvs/1617/1617_outcomes.csv", encoding = 'latin-1')
regular_season = pd.read_csv("../csvs/kaggle/regular_season_stats.csv", encoding = 'latin-1')

pd.set_option('display.max_rows',1755)

# adjust data sets to only 2014 and later
seeds = seeds[seeds.Season > 2013]
regions = regions[regions.Season > 2013]
reg_season = reg_season[reg_season.Season > 2013]

In [2]:
# Adds the column TeamName to the seeds dataframe
names = []
for team in seeds.TeamID:
    names.append(team_names['TeamName'][team_names['TeamID']==team].values[0])
seeds['TeamName'] = names

In [3]:
# initialize arrays
next_round           = []
roundof32            = []
sweet16              = []
elite8               = []
final4               = []
finals               = []
ncaa_winner          = []
roundof32_actual     = []
sweet16_actual       = []
elite8_actual        = []
final4_actual        = []
finals_actual        = []
ncaa_winner_actual   = []

tourney_order = ['W01','W16','W08','W09','W05','W12','W04','W13','W06','W11','W03','W14','W07','W10','W02','W15',
                 'X01','X16','X08','X09','X05','X12','X04','X13','X06','X11','X03','X14','X07','X10','X02','X15',
                 'Y01','Y16','Y08','Y09','Y05','Y12','Y04','Y13','Y06','Y11','Y03','Y14','Y07','Y10','Y02','Y15',
                 'Z01','Z16','Z08','Z09','Z05','Z12','Z04','Z13','Z06','Z11','Z03','Z14','Z07','Z10','Z02','Z15']

In [4]:
for i in range(0,32):
    roundof32_actual.append(outcomes_14['Round32'].values[i])
for i in range(0,16):
    sweet16_actual.append(outcomes_14['Round16'].values[i])
for i in range(0,8):
    elite8_actual.append(outcomes_14['Round8'].values[i])
for i in range(0,4):
    final4_actual.append(outcomes_14['Round4'].values[i])
for i in range(0,2):
    finals_actual.append(outcomes_14['Round2'].values[i])
for i in range(0,1):
    ncaa_winner_actual.append(outcomes_14['Round1'].values[i])
    
best_points = 0

In [5]:
def prediction(team1, team2, i1, i2, season):
    t1 = get_TeamID(team1, season)
    t2 = get_TeamID(team2, season)
    
    t1_stat = get_Stat(t1, season, i1)*get_Stat(t1, season, i2)
    t2_stat = get_Stat(t2, season, i1)*get_Stat(t1, season, i2)
    
    if t1_stat>t2_stat:
        return team1, team2, 0
    else:
        return team2, team1, 1

def array_filler(num_teams):
    if   num_teams == 64:
        roundof32.append(winner)
    elif num_teams == 32:
        sweet16.append(winner)
    elif num_teams == 16:
        elite8.append(winner)
    elif num_teams == 8:
        final4.append(winner)
    elif num_teams == 4:
        finals.append(winner)
    elif num_teams == 2:
        ncaa_winner.append(winner)
        
def getName(team, season):
    print("getName")
    return seeds['TeamName'][(seeds['Seed'] == team)&(seeds['Season']==season)].values[0]

def get_TeamID(team, season):
    return seeds['TeamID'][(seeds['Seed'] == team)&(seeds['Season']==season)].values[0]  

def get_Stat(team, season, indicator):
    return regular_season[(regular_season['TeamID']==team) & (regular_season['Season']==season)][indicator].values[0]

season = 2014

for indicator1 in regular_season.iloc[:, 32:]: # iloc - integer locate - all rows, columns 3 to end
    for indicator2 in regular_season.iloc[:, 32:]:

        #resets array
        roundof32, sweet16, elite8, final4, finals, ncaa_winner = [],[],[],[],[],[]
        next_round                                              = tourney_order[:]

        #print("It is this indicator ----> " + indicator)
        for round_num in range(0,6):        
            num_teams = 2**(6-round_num)
            #print("----------- Number of Teams left: %s" % (num_teams))
            for i in range(0, num_teams, 2):

                team1 = next_round[i]
                team2 = next_round[i+1]

                winner, loser, which = prediction(team1, team2,indicator1, indicator2, season)
                next_round.append(next_round[i+which])


                #print('%s beat %s to advance to the round of %s' % 
                      #(getName(winner,season),getName(loser,season), int(num_teams/2)))

                array_filler(num_teams)

            # splits the list so that the only elements left are teams still alive
            # after a Region the loop next_round will be empty

            del next_round[0:num_teams]
        games_correct = 0
        points = 0
        for i in range(0,32):
            if roundof32_actual[i] == roundof32[i]:
                points += 1
                games_correct += 1
        for i in range(0,16):
            if sweet16_actual[i] == sweet16[i]:
                points += 2
                games_correct += 1
        for i in range(0,8):
            if elite8_actual[i] == elite8[i]:
                points += 3 
                games_correct += 1
        for i in range(0,4):
            if final4_actual[i] == final4[i]:
                points += 4
                games_correct += 1
        for i in range(0,2):
            if finals_actual[i] == finals[i]:
                points += 6
                games_correct += 1
        for i in range(0,1):
            if ncaa_winner_actual[i] == ncaa_winner[i]:
                points += 8
                games_correct += 1
        #print("points for this indictor ---->" + str(points))        
        if points > best_points: 
            best_points = points
            best_games = games_correct
            best_indicator = indicator1 + " " +indicator2
        
print("The best indicator is: %s" % best_indicator)
print("Max total points is equal to: %d" % 120)
print("Our Total points is equal to: %d" % best_points)
print("We predicted %d games correct out of %d games!" % (best_games, 63) )
print("This comes out to be about %d%% of all games" % round(best_games*100/63) )

The best indicator is: OPF OStl
Max total points is equal to: 120
Our Total points is equal to: 22
We predicted 16 games correct out of 63 games!
This comes out to be about 25% of all games
